In [1]:
# Final Version

In [2]:
import pandas as pd
import numpy as np
from pulp import * 
import math
from datetime import datetime


# Copy and paste one of the scenarios listed below, and then Kernel > Restart & Run All
SCENARIO = 'REAL_DATA'

#REAL_DATA -- This is actual data from nurses for the 10/24 to 12/4 time period.
#SAME_REQUESTS -- All nurses have selected Day1,Day2, Day3 as their 3 shifts.  0 Vac, 0 Unavail
#VACATION_DAY1 -- Week1, All nurses have selected DAY1 to be on vacation, and work Day2,3. 
#The following weeks, all want to work Day1,2,3.



In [3]:
#IMPORT NURSES REQUIRED (DEMAND) ; 
nurse_demand = pd.read_excel (r'NurseDemandF.xlsx')
nurse_demand=nurse_demand[nurse_demand['SHIFT']=='DAYSHIFT']
nurse_demand=nurse_demand[['DAYNUM_LABEL', 'NURSES_REQUIRED']]
#print(nurse_demand)


In [4]:
#NURSES REQUIRED CONVERT DF TO DICT
nurse_demand_dict=dict(zip(nurse_demand.DAYNUM_LABEL, nurse_demand.NURSES_REQUIRED))
print(nurse_demand_dict)

{1: 4, 2: 3, 3: 3, 4: 3, 5: 3, 6: 4, 7: 4, 8: 4, 9: 3, 10: 3, 11: 3, 12: 3, 13: 4, 14: 4, 15: 4, 16: 3, 17: 3, 18: 3, 19: 3, 20: 4, 21: 4, 22: 4, 23: 3, 24: 3, 25: 3, 26: 3, 27: 4, 28: 4, 29: 4, 30: 3, 31: 3, 32: 3, 33: 3, 34: 4, 35: 4, 36: 4, 37: 3, 38: 3, 39: 3, 40: 3, 41: 4, 42: 4}


In [5]:
#NURSES REQUESTS: IMPORT 
NURSE_SHIFT_REQUESTS_ORIG = pd.read_excel(r'NurseShiftRequests6WksF.xlsx')

# FILTER FOR SCENARIO
NURSE_SHIFT_REQUESTS=NURSE_SHIFT_REQUESTS_ORIG[NURSE_SHIFT_REQUESTS_ORIG['SCENARIO']==SCENARIO].copy()
#print(NURSE_SHIFT_REQUESTS)

# convert EMPLOYEEID to string
NURSE_SHIFT_REQUESTS['EMPLOYEEID']= NURSE_SHIFT_REQUESTS['EMPLOYEEID'].map(str)

# get just the cols we need
NURSE_SHIFT_REQUESTS=NURSE_SHIFT_REQUESTS[NURSE_SHIFT_REQUESTS.columns[~NURSE_SHIFT_REQUESTS_ORIG.columns.isin(['SCENARIO','JOB','NURSEGROUP','SHIFTABBR'])]]
#print(NURSE_SHIFT_REQUESTS)

# MAKE EMPID THEINDEX
NURSE_SHIFT_REQUESTS.set_index('EMPLOYEEID', inplace=True)
#print(NURSE_SHIFT_REQUESTS)

#CONVERT DF TO DICT
nurse_shift_req_dictnew=NURSE_SHIFT_REQUESTS.to_dict()
#print(nurse_shift_req_dictnew)

In [6]:
# NURSE LIST -- NEED LIST TO LOOP OVER  

nurse_list = NURSE_SHIFT_REQUESTS.index.tolist()
#print(nurse_list)



In [7]:
#NURSE VAC -- GET DATA AND CONVERT TO DICT
NURSES_VAC = pd.read_excel (r'NurseVacation6WksF.xlsx')

# FILTER FOR SCENARIO
NURSES_VAC=NURSES_VAC[NURSES_VAC['SCENARIO']==SCENARIO].copy()


NURSES_VAC['EMPLOYEEID']= NURSES_VAC['EMPLOYEEID'].map(str)
NURSES_VAC=NURSES_VAC[NURSES_VAC.columns[~NURSES_VAC.columns.isin(['SCENARIO','JOB','NURSEGROUP','SHIFTABBR'])]]

# MAKE EMPID THEINDEX
NURSES_VAC.set_index('EMPLOYEEID', inplace=True)

#CONVERT DF TO DICT
nurse_vac_dictnew=NURSES_VAC.to_dict()
#print(nurse_vac_dictnew)

In [8]:
#NURSE UNAVAIL -- GET DATA AND CONVERT TO DICT
NURSES_UNAVAIL = pd.read_excel (r'NurseUnavailable6WksF.xlsx')

# FILTER FOR SCENARIO
NURSES_UNAVAIL=NURSES_UNAVAIL[NURSES_UNAVAIL['SCENARIO']==SCENARIO].copy()

NURSES_UNAVAIL['EMPLOYEEID']= NURSES_UNAVAIL['EMPLOYEEID'].map(str)
NURSES_UNAVAIL=NURSES_UNAVAIL[NURSES_UNAVAIL.columns[~NURSES_UNAVAIL.columns.isin(['SCENARIO','JOB','NURSEGROUP','SHIFTABBR'])]]

# MAKE EMPID THEINDEX
NURSES_UNAVAIL.set_index('EMPLOYEEID', inplace=True)

#NURSE UNAVAIL:  CONVERT DF TO DICT
nurse_unavailable_dictnew=NURSES_UNAVAIL.to_dict()
#print(nurse_unavailable_dictnew)


In [9]:
# NURSE GROUPS -- get list of empid's assigned to each group (df), and then put in dict via grouping series
# FILTER FOR SCENARIO
NURSE_SHIFT_REQUESTS_ORIG=NURSE_SHIFT_REQUESTS_ORIG[NURSE_SHIFT_REQUESTS_ORIG['SCENARIO']==SCENARIO].copy()
#print(NURSE_SHIFT_REQUESTS)

# convert EMPLOYEEID to string
NURSE_SHIFT_REQUESTS_ORIG['EMPLOYEEID']= NURSE_SHIFT_REQUESTS_ORIG['EMPLOYEEID'].map(str)
nurse_group = NURSE_SHIFT_REQUESTS_ORIG[['NURSEGROUP', 'EMPLOYEEID']]
#print(nurse_group)

nurse_group=nurse_group.groupby('NURSEGROUP')['EMPLOYEEID'].apply(list)
#print(nurse_group)
#print(type(nurse_group))

nurse_group_dict=nurse_group.to_dict()
#print(nurse_group_dict)



In [10]:
# NURSE POINT DICT
nurse_point_dict =  {'12295':10000, '31300':10000, '31407':10000, '38368':10000, '59561':10000 , '14581':5000, '36587':5000, '105865':5000,'18182':2500, '99263':2500   }

#print(nurse_point_dict['12295'])



In [11]:
# NURSE HISTORY DICT -- IF NURSE SHUFFLED >3X IN PREV SCHEDULING PERIOD TRIPLE POINTS
nurse_history_dict = {'12295':1, '31300':1, '31407':1, '38368':1, '59561':1 , '14581':1, '36587':1, '105865':3,'18182':1, '99263':1   }

In [12]:
# JUNIOR NURSES / SENIOR NURSES
junior_nurse_list = ['99263','14581']
senior_nurse_list = ['12295',  '18182', '31300', '31407', '36587', '38368', '59561',  '105865']

In [13]:
# DAYS LIST -- need a list of days to loop over.  There are col headings in NURSE_SHIFT_REQUESTS_ORIG so need 
# to get into list
# remove cols in df that are not like 'DayX', then convert the series to a list
daylistdf = NURSE_SHIFT_REQUESTS_ORIG[NURSE_SHIFT_REQUESTS_ORIG.columns[~NURSE_SHIFT_REQUESTS_ORIG.columns.isin(['SCENARIO','EMPLOYEEID', 'JOB','NURSEGROUP','SHIFTABBR'])]]

#print(daylist)
day_list = list(daylistdf.columns)
#print(day_list)


In [14]:
# MONDAYS:  Identify days that are Mondays; Nurses must work 2 Mndays
Mon = [2, 9, 16, 23, 30, 37 ]

In [15]:
# FRIDAYS: Identify days that are Fridays; Nurses must work 2 
Fri = [6, 13, 20, 27, 34, 41 ]

In [16]:
# SATURDAYS: Identify days that are SATURDAYS; Nurses must work 2 
Sat = [7, 14, 21, 28, 35, 42 ]

In [17]:
# SUNDAYS: Identify days that are SUNDAYS; Nurses must work 2 
Sun = [1, 8, 15, 22, 29, 36 ]

In [18]:
WEEK = ['Week_1', 'Week_2', 'Week_3', 'Week_4', 'Week_5', 'Week_6']

In [19]:
# Map days to weeks
WEEK_DAY_MAP ={ 
'Week_1': [1,  2, 3, 4, 5, 6, 7], 
'Week_2': [8,  9, 10,11, 12, 13, 14],
'Week_3': [15, 16, 17,18, 19, 20, 21],
'Week_4': [22, 23, 24,25, 26, 27, 28], 
'Week_5': [29, 30, 31,32, 33, 34, 35],  
'Week_6': [36, 37, 38,39, 40, 41, 42]  
}


In [20]:
# DECISION VARIABLE:  NURSE ASSIGNED SHIFT, YES/NO
X=pulp.LpVariable.dicts('AssignToWork',[(i,j) for i in day_list
                                              for j in nurse_list], 0,1,LpBinary)

In [21]:
# PROBLEM INSTANTIATION
# 
prob=LpProblem("NurseSchedOpt", LpMaximize)

In [22]:
# OBJECTIVE FUNCTION
# Reward shifts that nurses self scheduled and penalize scheduling them when they requested vac or unavil time

prob+=lpSum(nurse_history_dict[j]*nurse_point_dict[j]*nurse_shift_req_dictnew[i][j]*X[(i,j)] - nurse_history_dict[j]*nurse_point_dict[j]*nurse_vac_dictnew[i][j]*X[(i,j)] - nurse_history_dict[j]*nurse_point_dict[j]*nurse_unavailable_dictnew[i][j]*X[(i,j)] +  X[(i,j)] for i in day_list for j in nurse_list ) 



In [23]:
# CONSTRAINT:  for each day, number of nurses must be >= demand
for i in day_list:
    prob+=lpSum(X[(i,j)] for j in nurse_list ) >= nurse_demand_dict[i]

In [24]:
# CONSTRAINT: for each nurse, core hours must be at least 3 shifts per week for vac and shift work

for j in nurse_list:
    for w in WEEK:
              
        # need to handle when X =1, V=1 do not double count
        prob+= lpSum(   X[i,j] +  nurse_vac_dictnew[i][j]  - X[i,j]*nurse_vac_dictnew[i][j] for i in WEEK_DAY_MAP[w] )  <=3

In [25]:
# CONSTRAINT: each nurse must work 2 Mondays  
for j in nurse_list:
     prob+=lpSum(X[i,j]  for i in day_list if i in Mon   ) >=2


In [26]:
# CONSTRAINT: each nurse must work 2 Fridays   
for j in nurse_list:
     prob+=lpSum(X[i,j]  for i in day_list if i in Fri   ) >=2

In [27]:
# CONSTRAINT: each nurse must work 2 Saturdays   
for j in nurse_list:
     prob+=lpSum(X[i,j]  for i in day_list if i in Sat   ) >=2

In [28]:
# CONSTRAINT: each nurse must work 2 Sundays   
for j in nurse_list:
     prob+=lpSum(X[i,j]  for i in day_list if i in Sun   ) >=2

In [29]:
# CONSTRAINT: On any given day, must have ratio of 2 senior nurses to every one jr nurses

for i in day_list:
    lhs = lpSum( X[i,j] for j in senior_nurse_list )
    rhs = lpSum( X[i,j] for j in junior_nurse_list ) 
    prob+=  lhs >= 2* rhs 


In [30]:
# Solve
prob.solve()
print("Status:", LpStatus[prob.status])


Status: Optimal


In [31]:
# Print all the details
#print(prob)

In [32]:
status = prob.status
# 1.Optimal
# 2.Not Solved
# 3.Infeasible
# 4.Unbounded
# 5.Undefined


if status ==1 or status == 2 or status == 3 or status ==4  or status ==5 :
    output = []
    for variable in prob.variables():
        variablenameclean1 = variable.name.replace("(", "")
        variablenameclean2 = variablenameclean1.replace(")", "")
        variablenameclean3 = variablenameclean2.replace("'", "")
        variablenameclean4 = variablenameclean3.replace(",", "")
        varstring = variablenameclean4 +':' + str(variable.varValue)
        output.append(varstring)
    

cols = ['original']          
df_output = pd.DataFrame(output, columns = cols)
#print(df_output)



original = df_output['original']

df_output[['variablename', 'variablevalue']] = original.str.split(":",n=1, expand=True)
#print(df_output)


df_output['variablevalue'] = df_output['variablevalue'].str.strip()
df_output['variablevalue'] = pd.to_numeric(df_output['variablevalue'])

variablename = df_output['variablename']
df_output[['variablename', 'day', 'EMPLOYEEID']] = variablename.str.split("_", n=2, expand=True)
#print(df_output)

df_output['day'] = df_output['day'].astype(int)
#print(df_output)


df_output.sort_values(by=['variablename','EMPLOYEEID','day'], inplace=True)

# get df for AssignToWork
df_output = df_output[df_output['variablename']=="AssignToWork"]  
df_output.to_excel("df_output.xlsx")
#print(df_output)





        

In [33]:
# Print Optimal Solution
print("The max points for this scheduling period is:", value(prob.objective))

The max points for this scheduling period is: 820174.0


In [34]:
#MERGE NURSES AND THEIR GROUPS TO OUTPUT FROM OPT

# GET DF OF NURSES AND GROUPS
nurse_group_df = NURSE_SHIFT_REQUESTS_ORIG[['NURSEGROUP', 'EMPLOYEEID']]
#print(nurse_group_df)

# MERGE 
df_output_w_group = pd.merge(df_output , nurse_group_df, how = 'left',  on = 'EMPLOYEEID' )

# print(df_output_w_group)





In [35]:
# Pivot results; create index

# create new col called empid_group -- we will use for index
df_output_w_group['group_empid'] = df_output_w_group['NURSEGROUP'] + "_" + df_output_w_group['EMPLOYEEID']
#print(df_output_w_group)

# sort
df_output_w_group.sort_values(by=['group_empid', 'day'], inplace=True)
#print(df_output_w_group)

df_output_pivot=df_output_w_group.pivot(index='group_empid', columns='day', values='variablevalue')

df_output_pivot_final=df_output_pivot.add_prefix('Day')
print(df_output_pivot_final)

day             Day1  Day2  Day3  Day4  Day5  Day6  Day7  Day8  Day9  Day10  \
group_empid                                                                   
Group_1_12295    0.0   1.0   1.0   0.0   0.0   0.0   0.0   0.0   0.0    0.0   
Group_1_31300    0.0   0.0   0.0   1.0   0.0   0.0   1.0   1.0   0.0    0.0   
Group_1_31407    0.0   0.0   0.0   0.0   1.0   0.0   1.0   1.0   1.0    1.0   
Group_1_38368    0.0   1.0   0.0   0.0   0.0   1.0   0.0   1.0   1.0    1.0   
Group_1_59561    1.0   1.0   1.0   0.0   0.0   0.0   0.0   1.0   1.0    1.0   
Group_2_105865   0.0   0.0   1.0   0.0   0.0   1.0   0.0   1.0   1.0    1.0   
Group_2_14581    0.0   0.0   0.0   0.0   1.0   0.0   1.0   1.0   0.0    0.0   
Group_2_36587    1.0   0.0   0.0   1.0   0.0   1.0   0.0   0.0   0.0    0.0   
Group_3_18182    1.0   0.0   0.0   0.0   1.0   0.0   1.0   1.0   0.0    0.0   
Group_3_99263    1.0   0.0   0.0   1.0   0.0   1.0   0.0   0.0   1.0    0.0   

day             ...  Day33  Day34  Day35  Day36  Da

In [36]:
# GET NURSE REQUESTS IN SAME STRUCTURE AS OUTPUT SO CAN COMPARE

NURSE_SHIFT_REQUESTS_ORIG['group_empid'] = NURSE_SHIFT_REQUESTS_ORIG['NURSEGROUP'] + "_" + NURSE_SHIFT_REQUESTS_ORIG['EMPLOYEEID']

# get the cols that we need
nurse_shift_request_compare=NURSE_SHIFT_REQUESTS_ORIG[NURSE_SHIFT_REQUESTS_ORIG.columns[~NURSE_SHIFT_REQUESTS_ORIG.columns.isin(['SCENARIO','EMPLOYEEID','JOB','NURSEGROUP','SHIFTABBR'])]]
#print(nurse_shift_request_compare)

# create index
nurse_shift_request_compare.set_index('group_empid', inplace = True)

nurse_shift_request_compare=nurse_shift_request_compare.add_prefix('Day')
nurse_shift_request_compare_final = nurse_shift_request_compare.copy()   

#print(nurse_shift_request_compare_final)

# sort
nurse_shift_request_compare_final.sort_index(axis=0,inplace=True)
print(nurse_shift_request_compare_final)


                Day1  Day2  Day3  Day4  Day5  Day6  Day7  Day8  Day9  Day10  \
group_empid                                                                   
Group_1_12295      0     1     1     0     0     0     0     1     0      1   
Group_1_31300      0     1     1     0     0     0     0     1     1      1   
Group_1_31407      0     1     1     0     0     0     0     1     1      1   
Group_1_38368      0     1     1     0     0     0     0     1     1      1   
Group_1_59561      0     1     1     0     0     0     0     1     1      1   
Group_2_105865     0     1     1     0     0     0     0     1     1      1   
Group_2_14581      0     1     1     0     0     0     0     1     1      1   
Group_2_36587      0     1     1     0     0     0     0     1     1      1   
Group_3_18182      0     1     1     0     0     0     0     1     1      1   
Group_3_99263      0     1     1     0     0     0     0     1     1      1   

                ...  Day33  Day34  Day35  Day36  Da

In [37]:
# COMPARE THESE TWO DF'S
# subtract requests from final version  to see the difference
df_output_pivot_final.subtract(nurse_shift_request_compare_final, axis='columns', level=None, fill_value=None)


day             Day1  Day2  Day3  Day4  Day5  Day6  Day7  Day8  Day9  Day10  \
group_empid                                                                   
Group_1_12295    0.0   0.0   0.0   0.0   0.0   0.0   0.0  -1.0   0.0   -1.0   
Group_1_31300    0.0  -1.0  -1.0   1.0   0.0   0.0   1.0   0.0  -1.0   -1.0   
Group_1_31407    0.0  -1.0  -1.0   0.0   1.0   0.0   1.0   0.0   0.0    0.0   
Group_1_38368    0.0   0.0  -1.0   0.0   0.0   1.0   0.0   0.0   0.0    0.0   
Group_1_59561    1.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0    0.0   
Group_2_105865   0.0  -1.0   0.0   0.0   0.0   1.0   0.0   0.0   0.0    0.0   
Group_2_14581    0.0  -1.0  -1.0   0.0   1.0   0.0   1.0   0.0  -1.0   -1.0   
Group_2_36587    1.0  -1.0  -1.0   1.0   0.0   1.0   0.0  -1.0  -1.0   -1.0   
Group_3_18182    1.0  -1.0  -1.0   0.0   1.0   0.0   1.0   0.0  -1.0   -1.0   
Group_3_99263    1.0  -1.0  -1.0   1.0   0.0   1.0   0.0  -1.0   0.0   -1.0   

day             ...  Day33  Day34  Day35  Day36  Day37  Day38  Day39  Day40  \
group_empid     ...                                                           
Group_1_12295   ...    1.0    0.0    0.0   -1.0    0.0    0.0    0.0    0.0   
Group_1_31300   ...    0.0    1.0    0.0    0.0    0.0    0.0    0.0    0.0   
Group_1_31407   ...    0.0    0.0    1.0    0.0   -1.0   -1.0    0.0    0.0   
Group_1_38368   ...    0.0    0.0    0.0    0.0    0.0   -1.0    0.0    0.0   
Group_1_59561   ...    0.0    1.0    0.0    0.0    0.0   -1.0    0.0    0.0   
Group_2_105865  ...    0.0    0.0    1.0    0.0    0.0    0.0    0.0    0.0   
Group_2_14581   ...    1.0    0.0    1.0   -1.0    0.0   -1.0    1.0    0.0   
Group_2_36587   ...    1.0    0.0    0.0   -1.0    0.0   -1.0    1.0    1.0   
Group_3_18182   ...    0.0    1.0    1.0   -1.0   -1.0   -1.0    1.0    1.0   
Group_3_99263   ...    0.0    1.0    0.0   -1.0    0.0   -1.0    0.0    1.0   

day             Day41  Day42  
group_empid                   
Group_1_12295     1.0    0.0  
Group_1_31300     0.0    0.0  
Group_1_31407     1.0    1.0  
Group_1_38368     1.0    0.0  
Group_1_59561     0.0    1.0  
Group_2_105865    0.0    0.0  
Group_2_14581     1.0    0.0  
Group_2_36587     0.0    0.0  
Group_3_18182     0.0    1.0  
Group_3_99263     0.0    1.0  

[10 rows x 42 columns]

In [38]:
# EXPORT SUBTRACTED DF

result = df_output_pivot_final.subtract(nurse_shift_request_compare_final, axis='columns', level=None, fill_value=None)

#print(result)
### Export result to C drive
now = datetime.now()
dt_string = now.strftime("%Y%m%d_%H%M")
#print(dt_string)

filename = "result_subtract_" + SCENARIO+"_" + dt_string + ".xlsx"
#type(filename)
#print(filename)
result.to_excel(filename)


In [39]:
# VALIDATION 
# Sum each row to make sure each nurse has 18 shifts (except if nurse has vacation..31407(1day) and 59561(3days) )

prefix_dow = 'Day'
Sun = [prefix_dow + str(x) for x in Sun]
Mon = [prefix_dow + str(x) for x in Mon]
Fri = [prefix_dow + str(x) for x in Fri]
Sat = [prefix_dow + str(x) for x in Sat]
day_list = [prefix_dow + str(x) for x in day_list]
demand_3nurse_list =  [prefix_dow + str(x) for x in [2,3,4,5,9,10,11,12,16,17,18,19,23,24,25,26,30,31,32,33,37,38,39,40]]
demand_4nurse_list =  [prefix_dow + str(x) for x in [1,6,7,8,13,14,15,20,21,22,27,28,29,34,35,36,41,42]]
#print(Mon)
#print(Fri)
#print(Sat)
#print(Sun)
#print(day_list)
#print(three_nurse_demand)

df_row_validation = df_output_pivot_final.copy()
#print(df_row_validation)

df_row_validation["sum_shifts"] = df_row_validation.sum(axis=1)
#print(df_row_validation)

# Sum Mondays
df_row_validation["sum_Mon"] = df_row_validation[Mon].sum(axis=1)

# Sum Fridays
df_row_validation["sum_Fri"] = df_row_validation[Fri].sum(axis=1)

# Sum Saturdays
df_row_validation["sum_Sat"] = df_row_validation[Sat].sum(axis=1)

# Sum Sundays
df_row_validation["sum_Sun"] = df_row_validation[Sun].sum(axis=1)

# print these new summary variables created above
print(df_row_validation.iloc[:,42:47])

# column validation
df_col_validation = df_output_pivot_final.copy()
#print(df_col_validation)

# do nurses working meet the demand? 
demand_3nurse_validation = min(df_col_validation[demand_3nurse_list].sum(axis=0) )
print("Min. #nurses working per model for days when demand must be >=3 :" ,  demand_3nurse_validation)

demand_4nurse_validation = min(df_col_validation[demand_4nurse_list].sum(axis=0) )
print("Min. #nurses working per model for days when demand must be >=4 :" ,  demand_4nurse_validation)

# is there a ratio of 2 : 1 senior to jr nurses?
jrfilt = df_col_validation.loc[['Group_2_14581','Group_3_99263' ]] 
jrfilt_daysum = jrfilt.sum(axis=0, numeric_only=True)
jrfilt_daysum = jrfilt_daysum[jrfilt_daysum>=2]
print(jrfilt_daysum)

srfilt = df_col_validation.loc[['Group_1_12295','Group_1_31300', 'Group_1_31407' , 'Group_1_38368', 'Group_1_59561','Group_2_105865', 'Group_2_36587', 'Group_3_18182' ]] 
srfilt_daysum = srfilt.sum(axis=0, numeric_only=True)
srfilt_daysum = srfilt_daysum[srfilt_daysum>=4]
print(srfilt_daysum)   
    

day             sum_shifts  sum_Mon  sum_Fri  sum_Sat  sum_Sun
group_empid                                                   
Group_1_12295         17.0      5.0      2.0      2.0      3.0
Group_1_31300         17.0      2.0      3.0      2.0      4.0
Group_1_31407         17.0      3.0      2.0      3.0      4.0
Group_1_38368         17.0      6.0      2.0      2.0      3.0
Group_1_59561         18.0      5.0      3.0      2.0      4.0
Group_2_105865        17.0      4.0      2.0      2.0      4.0
Group_2_14581         17.0      4.0      2.0      3.0      3.0
Group_2_36587         18.0      2.0      2.0      2.0      2.0
Group_3_18182         18.0      2.0      2.0      3.0      2.0
Group_3_99263         18.0      2.0      4.0      3.0      2.0
Min. #nurses working per model for days when demand must be >=3 : 3.0
Min. #nurses working per model for days when demand must be >=4 : 4.0
day
Day37    2.0
dtype: float64
day
Day8     6.0
Day9     4.0
Day10    4.0
Day15    4.0
Day16    5.0
Day

In [40]:
### Export result to C drive
now = datetime.now()
dt_string = now.strftime("%Y%m%d_%H%M")
#print(dt_string)

filename = "output_pvt_" + SCENARIO+"_" + dt_string + ".xlsx"
#type(filename)
#print(filename)
df_output_pivot.to_excel(filename)